# Customer Segmentation

RFM= 
RECENCY   ---->  HOW RECENTLY CUSTOMER PURCHASED,
FREQUENCY ----> HOW OFTEN THEY PURCHASE,
MONETARY  ----> HOW MUCH THEY SPEND

In [2]:
import sqlite3
import pandas as pd
conn=sqlite3.connect("retail.db")

In [3]:
pd.read_sql("SELECT * FROM transactions",conn)

,OrderLineID,OrderID,ProductID,Qty,UnitPrice,LineDiscount$,LineCost$,LineRevenue$
0,OL0000001,O000001,P001689,2,323.19,45.22,363.40,601.16
1,OL0000002,O000001,P000676,3,34.66,7.27,59.13,96.71
2,OL0000003,O000001,P000972,1,76.08,5.32,45.59,70.76
3,OL0000004,O000001,P001872,3,45.45,9.54,93.42,126.81
4,OL0000005,O000001,P001669,1,78.55,5.50,50.67,73.05
...,...,...,...,...,...,...,...,...
182133,OL0182134,O044999,P002620,1,45.48,0.00,34.20,45.48
182134,OL0182135,O044999,P003262,3,196.63,196.63,360.81,393.26
182135,OL0182136,O045000,P000419,1,184.75,56.56,104.75,128.19
182136,OL0182137,O045000,P002702,2,137.51,84.20,145.34,190.82


In [5]:
query=""" PRAGMA table_info(transactions);"""

pd.read_sql(query,conn)

,cid,name,type,notnull,dflt_value,pk
0,0,OrderLineID,TEXT,0,None,0
1,1,OrderID,TEXT,0,None,0
2,2,ProductID,TEXT,0,None,0
3,3,Qty,INTEGER,0,None,0
4,4,UnitPrice,REAL,0,None,0
5,5,LineDiscount$,REAL,0,None,0
6,6,LineCost$,REAL,0,None,0
7,7,LineRevenue$,REAL,0,None,0


Basket analysis
Promotion targeting
Order value prediction
This is called Order Value Segmentation.

In [ ]:
# OrderID ≈ Customer session / purchase event

query="""
select 
        OrderID,
        Count(DISTINCT ProductID) As unique_products,
        Sum(Qty) as Total_items,
        sum(LineRevenue$) as Order_revenue,
        sum(LineDiscount$) as total_discount
    from transactions
    group by OrderID

"""
orderdf=pd.read_sql(query,conn)
orderdf.head()

,OrderID,unique_products,Total_items,Order_revenue,total_discount
0,O000001,6,13,1332.40,100.23
1,O000002,4,7,760.41,203.33
2,O000003,3,6,1296.73,38.67
3,O000004,3,8,608.23,276.20
4,O000005,4,7,1031.90,30.32


In [7]:
orderdf.describe()

,unique_products,Total_items,Order_revenue,total_discount
count,45000.000000,45000.000000,45000.000000,45000.000000
mean,4.045711,7.289667,996.394873,233.813866
std,1.312408,2.944176,531.642096,220.793369
min,1.000000,1.000000,3.030000,0.000000
25%,3.000000,5.000000,610.897500,64.550000
50%,4.000000,7.000000,928.720000,158.835000
75%,5.000000,9.000000,1305.847500,345.142500
max,6.000000,19.000000,3989.440000,2045.070000


In [ ]:
# Step- Create value-based segments

orderdf['value_seg']=pd.qcut(orderdf['Order_revenue'],q=4,labels=['low','med','high','premium'])
orderdf['value_seg'].value_counts()

value_seg
med        11251
low        11250
premium    11250
high       11249
Name: count, dtype: int64

In [10]:
orderdf['basket_type'] = pd.cut(
    orderdf['unique_products'],
    bins=[0,1,3,5,100],
    labels=['Single-item','Small','Medium','Large']
)

orderdf[['value_seg','basket_type']].head()


,value_seg,basket_type
0,premium,Large
1,med,Medium
2,high,Small
3,low,Small
4,high,Medium


In [35]:
orderdf.groupby(['value_seg','basket_type']).agg(
    avg_revenue=('Order_revenue','mean'),
    avg_items=('Total_items','mean'),
    avg_discount=('total_discount','mean'),
    orders=('OrderID','count')
)

/var/folders/kv/02xnk4yx4q94gr1r59m8rd140000gn/T/ipykernel_82789/106487853.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  orderdf.groupby(['value_seg','basket_type']).agg(


avg_revenue  avg_items  avg_discount  orders
value_seg basket_type                                              
low       Single-item   209.033034   1.686286     53.036331    1750
          Small         395.382656   4.090274    118.006299    6303
          Medium        483.904547   6.406149    161.154896    3090
          Large         538.456262   9.121495    190.724112     107
med       Single-item   728.112245   3.081633    116.740612      98
          Small         750.883441   5.158223    192.760480    3691
          Medium        779.351773   7.181927    244.551840    6695
          Large         802.975541   9.251630    286.501173     767
high      Single-item  1046.115714   3.571429     55.195000      14
          Small        1082.423883   6.093564    210.435949    1849
          Medium       1103.763998   8.067719    292.481027    7413
          Large        1125.891602  10.095286    361.123457    1973
premium   Single-item  1427.260000   4.500000     78.005000       2
          Small        1546.156516   7.196891    188.680557     772
          Medium       1680.137160   9.484280    272.846297    6552
          Large        1816.028682  11.535423    352.361504    3924

### Fature engineering


In [13]:
model_df=orderdf.copy()

In [14]:
model_df

,OrderID,unique_products,Total_items,Order_revenue,total_discount,value_seg,basket_type
0,O000001,6,13,1332.40,100.23,premium,Large
1,O000002,4,7,760.41,203.33,med,Medium
2,O000003,3,6,1296.73,38.67,high,Small
3,O000004,3,8,608.23,276.20,low,Small
4,O000005,4,7,1031.90,30.32,high,Medium
...,...,...,...,...,...,...,...
44995,O044996,6,16,1289.18,750.21,high,Large
44996,O044997,6,8,1098.31,416.59,high,Large
44997,O044998,5,9,1471.25,168.60,premium,Medium
44998,O044999,6,12,1102.56,575.17,high,Large


In [ ]:
query=""" PRAGMA table_info(transactions);"""
pd.read_sql(query,conn)

,cid,name,type,notnull,dflt_value,pk
0,0,OrderLineID,TEXT,0,None,0
1,1,OrderID,TEXT,0,None,0
2,2,ProductID,TEXT,0,None,0
3,3,Qty,INTEGER,0,None,0
4,4,UnitPrice,REAL,0,None,0
5,5,LineDiscount$,REAL,0,None,0
6,6,LineCost$,REAL,0,None,0
7,7,LineRevenue$,REAL,0,None,0


In [15]:
model_df.columns

Index(['OrderID', 'unique_products', 'Total_items', 'Order_revenue',
       'total_discount', 'value_seg', 'basket_type'],
      dtype='object')

In [16]:
# Add Profit(business-focused)

query="""

select 
    OrderID,
    sum(LineRevenue$) as revenue,
    sum(LineCost$) as cost
from transactions
group by OrderID
"""

In [18]:
profit_df=pd.read_sql(query,conn)

In [19]:

profit_df

,OrderID,revenue,cost
0,O000001,1332.40,813.90
1,O000002,760.41,526.61
2,O000003,1296.73,792.10
3,O000004,608.23,554.51
4,O000005,1031.90,665.33
...,...,...,...
44995,O044996,1289.18,1380.28
44996,O044997,1098.31,893.41
44997,O044998,1471.25,939.28
44998,O044999,1102.56,1033.13


In [20]:
profit_df['profit']=profit_df['revenue']-profit_df['cost']

In [21]:
model_df=model_df.merge(profit_df[['OrderID','profit']],on='OrderID')

In [24]:
model_df.head()

,OrderID,unique_products,Total_items,Order_revenue,total_discount,value_seg,basket_type,profit
0,O000001,6,13,1332.40,100.23,premium,Large,518.50
1,O000002,4,7,760.41,203.33,med,Medium,233.80
2,O000003,3,6,1296.73,38.67,high,Small,504.63
3,O000004,3,8,608.23,276.20,low,Small,53.72
4,O000005,4,7,1031.90,30.32,high,Medium,366.57


### Prediction

In [25]:
model_df.columns

Index(['OrderID', 'unique_products', 'Total_items', 'Order_revenue',
       'total_discount', 'value_seg', 'basket_type', 'profit'],
      dtype='object')

In [26]:
threshold=model_df['Order_revenue'].quantile(0.75)
model_df['high_value_order']=(model_df['Order_revenue']>=threshold.astype(int))
model_df['high_value_order'].value_counts()


high_value_order
False    33721
True     11279
Name: count, dtype: int64

In [27]:
features=[
    'Total_items',
    'unique_products',
    'total_discount',
    'profit'
]

X=model_df[features]
Y=model_df['high_value_order']

In [28]:
X

,Total_items,unique_products,total_discount,profit
0,13,6,100.23,518.50
1,7,4,203.33,233.80
2,6,3,38.67,504.63
3,8,3,276.20,53.72
4,7,4,30.32,366.57
...,...,...,...,...
44995,16,6,750.21,-91.10
44996,8,6,416.59,204.90
44997,9,5,168.60,531.97
44998,12,6,575.17,69.43


In [ ]:
# pip install scikit-learn

  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 31.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 41.7 MB/s eta 0:00:00a 0:00:01
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


In [32]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=.25,random_state=42,stratify=Y)
model=LogisticRegression(max_iter=100)
model.fit(x_train,y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [33]:
y_pred=model.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.96      0.97      0.96      8430
        True       0.91      0.87      0.89      2820

    accuracy                           0.94     11250
   macro avg       0.93      0.92      0.92     11250
weighted avg       0.94      0.94      0.94     11250



In [34]:
importance = pd.DataFrame({
    'feature': features,
    'coefficient': model.coef_[0]
}).sort_values(by='coefficient', ascending=False)

importance


,feature,coefficient
0,Total_items,0.454763
3,profit,0.026171
2,total_discount,0.013913
1,unique_products,-0.014053


### Validate

In [37]:
from sklearn.metrics import roc_auc_score

y_prob = model.predict_proba(x_test)[:,1]
roc_auc_score(y_test, y_prob)


0.98567703995356